<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Insights/blob/main/05-CSV_Data_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Interacting with a CSV Data

스마트서울 도시데이터 센서(S-DoT) 2분단위 환경정보
더보기
※  데이터 변환으로 2024년 7월 10일 이후 부터 이용 가능합니다.
### 서울시 전역(1100곳)에 설치된 S-DoT 복합 센서를 통해 2분 간격으로 수집되는 기온, 상대습도, 조도, 소음, 진동, 자외선, 풍향, 풍속, 흑구온도, 대기오염물질, 악취 등의 환경정보 데이터입니다.
### 서울 열린데이터광장에서는 동일 데이터로 1시간 단위 평균값으로 제공하고 있습니다.


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/1zcBjhPUcSU72Iik22GzLPyGtQwTrq3Qa/export?format=xlsx' -O wv-corelation.xlsx

In [ ]:
!pip install python-dotenv openai langchain langchain-core langchain-community langchain_experimental

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=<FILEID>'  -O ./.env

In [4]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [5]:
import os
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.0)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## Load the dataset

In [6]:
import os
import pandas as pd

# Load the Excel file
file_path = 'wv-corelation.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame to understand its structure
df.head()

,ds,wv,dp-pre,dp-med,co2,current,temperature,humidity
0,2023-09-25 10:12:00,8516.666667,164.666667,164.666667,502.750000,4.930000,164.666667,48.760
1,2023-09-25 10:13:00,8526.333333,153.666667,153.666667,507.000000,4.926667,153.666667,48.790
2,2023-09-25 10:14:00,8525.500000,176.666667,176.666667,505.666667,4.853333,176.666667,48.850
3,2023-09-25 10:15:00,8516.666667,159.000000,159.000000,495.750000,4.780000,159.000000,48.805
4,2023-09-25 10:16:00,8556.000000,160.666667,160.666667,499.250000,4.910000,160.666667,48.805


In [ ]:
# Convert the 'ds' column to datetime format
df['ds'] = pd.to_datetime(df['ds'])

# Set the 'ds' column as the index
df.set_index('ds', inplace=True)

# Resample the data to 1-hour intervals and calculate the mean for each interval
df_resampled = df.resample('1H').mean()

# Display the first few rows of the resampled DataFrame
print(df_resampled.head())
df = df_resampled

## Prepare the Langchain dataframe agent

In [8]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# Ensure you have your LLM instance created as `llm`

agent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, allow_dangerous_code=True)

# Now you can invoke the agent with a query
response = agent.invoke("what are the names of the columns?")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to access the dataframe's columns attribute
Action: python_repl_ast
Action Input: df.columnsIndex(['wv', 'dp-pre', 'dp-med', 'co2', 'current', 'temperature', 'humidity'], dtype='object') I now know the final answer
Final Answer: The names of the columns are 'wv', 'dp-pre', 'dp-med', 'co2', 'current', 'temperature', and 'humidity'.

> Finished chain.
{'input': 'what are the names of the columns?', 'output': "The names of the columns are 'wv', 'dp-pre', 'dp-med', 'co2', 'current', 'temperature', and 'humidity'."}


## Design your prompt and ask your question

In [9]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "Calculate and analyze the correlation of the 'wv' column with the remaining input columns of the 'target variable'."


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: I should set the pandas display options to show all the columns
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None)NameError: name 'pd' is not defined I should import pandas as pd first
Action: python_repl_ast
Action Input: import pandas as pd I should set the pandas display options to show all the columns
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None) I should get the column names
Action: python_repl_ast
Action Input: df.columnsIndex(['wv', 'dp-pre', 'dp-med', 'co2', 'current', 'temperature', 'humidity'], dtype='object') I should calculate the correlation of the 'wv' column with the remaining input columns of the 'target variable'
Action: python_repl_ast
Action Input: df.corr()['wv']wv             1.000000
dp-pre         0.073339
dp-med         0.073339
co2           -0.066431
current       -0.142634
temperature    0.073339
humidity       0.247938
Name: wv, dtype: float64 I 

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nCalculate and analyze the correlation of the \'wv\' column with the remaining input columns of the \'target variable\'.\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a s